In [ ]:
import adaptive
import nest_asyncio

adaptive.notebook_extension()
nest_asyncio.apply()

In [ ]:
from pathlib import Path
import json
import uuid

from openmc_model import find_tbr_hcpb


def find_tbr(x):

    breeder_percent_in_breeder_plus_multiplier_ratio, blanket_breeder_li6_enrichment = x
                           
    result = find_tbr_hcpb(breeder_percent_in_breeder_plus_multiplier_ratio,
                           blanket_breeder_li6_enrichment)

    result["sample"] = "adaptive"

    filename = "outputs/" + str(uuid.uuid4()) + ".json"
    Path(filename).parent.mkdir(parents=True, exist_ok=True)
    with open(filename, mode="w", encoding="utf-8") as f:
        json.dump(result, f, indent=4)

    return result["tbr"]


number_of_simulations = 4


print("running simulations with adaptive sampling")

learner = adaptive.Learner2D(find_tbr, bounds=[(0, 100), (0, 100)])

runner = adaptive.Runner(learner, ntasks=1, goal=lambda l: l.npoints > number_of_simulations)

# example goal setting for acceptable coverage error is also possible
# runner = adaptive.Runner(learner, ntasks=1, goal=lambda l: l.loss() < 0.01)

# runner.live_info()
# runner.live_plot()

runner.ioloop.run_until_complete(runner.task)

print("results saved in outputs folder")
